In [1]:
#Quantum Programming Homework 6: Bars and Stripes Classification
#Nathan Taylor     #0 1    stripes: 0 0  1 1   bars:  1 0   0 1
#05/05/22          #2 3             1 1  0 0          1 0   0 1
import cirq
import numpy as np
import random

In [2]:
#a) Parameter Shift
#Objective function: probability of finding 1
def simCircuit(oplist, counts):#Simulate the circuit
    sim = cirq.Simulator()
    circuit = cirq.Circuit(oplist)
    #print(circuit)
    sample = sim.run(circuit, repetitions=counts)
    statelist = str(sample)[2:].split(", ")#list of states for each qubit
    
    #format output
    L = len(statelist)#number of qubits
    state_dictionary = {}
    for c in range(counts):
        nstate = ""
        for q in range(L):
            nstate += str(statelist[q][c])
        if nstate in state_dictionary:
            state_dictionary[nstate] += 1
        else:
            state_dictionary[nstate] = 1
    stateArray = np.zeros([0, 2])
    for key, value in state_dictionary.items():
        stateArray = np.append(stateArray, [np.array([key, str(value/counts)])], axis=0)
    
    return stateArray

def objFunction(data):#Zexpectation Value
    L = len(data[0, 0])#number of qubits
    z_avg = np.zeros(L)#list of averages
    
    for row in data:
        for q in range(L):
            z_avg[q] += float(row[0][q]) * float(row[1])#multiply each qubit by the probability the state appeared
    return z_avg

def rxGate(ang, qubits):
    oplist = []
    oplist.append(cirq.rx(ang)(qubits[0]))
    oplist.append(cirq.measure(qubits[0]))
    return objFunction(simCircuit(oplist, 1000))

def singleShift(qubits, ang, count):
    A = 0.1#parameter shift
    F0 = rxGate(ang, qubits)[0]
    FA = rxGate(ang + A, qubits)[0]
    #print("F0:", F0)
    shiftAngle = ang + A*(FA-F0)
    if F0 < 0.99 and count > 0:
        count -= 1
        singleShift(qubits, shiftAngle, count)
    else:
        print("Fidelity:", F0)
        print("Found Angle:", ang)

In [3]:
qubits = cirq.LineQubit.range(4)
anc = cirq.NamedQubit("A")#0 for bars, 1 for stripes
singleShift(qubits, np.pi/2, 10000)

Fidelity: 0.99
Found Angle: 2.8489963267948903


In [4]:
#b) Parameterized Layers
def encodeCircuit(alist, plist, qubits, anc):
    gatelist = []
    side = []
    bars = [[1, 3], [0, 2]]
    stripes = [[2, 3], [0, 1]]
    
    goal = 1#Goal is the target state of objective
    choice = random.random()
    if random.random() < 0.5:
        side = random.choice(bars)
        goal = 0
    else:
        side = random.choice(stripes)

    #Initial Gates
    for i in side:
        gatelist.append(cirq.X(qubits[i]))
        
    #Single Qubit Rotations
    gatelist.append(plist[0](alist[0])(qubits[0]))
    gatelist.append(plist[1](alist[1])(qubits[1]))
    gatelist.append(plist[2](alist[2])(qubits[2]))
    gatelist.append(plist[3](alist[3])(qubits[3]))
    gatelist.append(plist[4](alist[4])(anc))
    
    #Entangling Gates - these were not chosen randomly, this directly encodes the problem into the ancilla
    #For the record, I came up with this idea and it is basically cheating
    gatelist.append(cirq.CNOT(qubits[0], qubits[1]))
    gatelist.append(cirq.CNOT(qubits[1], anc))#q1 is 0: 01 or 23, 1: 02 or 13s
    #0 1  stripes: 0 0  1 1   bars:  1 0   0 1
    #2 3           1 1  0 0          1 0   0 1
    
    #Readout Rotation
    gatelist.append(plist[5](alist[5])(anc))
    
    gatelist.append(cirq.measure(anc, key = 'z'))
    return objFunction(simCircuit(gatelist, 1000)), goal

In [5]:
#generate gates and initial angles
alist = np.zeros(6)
plist = []
for i in range(6):
    plist.append(random.choice([cirq.ry, cirq.rx]))#error types
    alist[i] = np.pi/random.randint(1, 21)

print(encodeCircuit(alist, plist, qubits, anc))

(array([0.12]), 1)


In [6]:
#c) The whole enchilada
def updateParameter(alist, plist, qubits, anc, rlim):
    shift = 0.1
    F0, goal = encodeCircuit(alist, plist, qubits, anc)
    F0 = abs(1-goal-F0)#if goal is 1, we want F0 to be 1, if goal is 0, we want it to be 0
    
    #if goal:
    #    print("Stripes F0:", round(F0[0],3))
    #else:
    #    print("Bars F0:", round(F0[0],3))
    #print("F0:", F0[0])
        
    FU = np.zeros(len(alist))#Were doing this for each angle separately I guess
    for i in range(len(alist)):
        temp_ang = np.copy(alist)
        temp_ang[i] += shift
        FU[i], gt = encodeCircuit(temp_ang, plist, qubits, anc)#the update circuit has the same initial state as F0
        FU[i] = abs(1-gt-FU[i])
    shiftedAngles = alist + shift*(FU-F0)#*rlim/1000#no idea about this step thing
    if F0 < 0.999 and rlim > 0:
        rlim -= 1
        return updateParameter(shiftedAngles, plist, qubits, anc, rlim)
    else:
        print("Fidelity:", F0)
        print("Angles:", alist)
        return alist

In [7]:
angles = updateParameter(alist, plist, qubits, anc, 2000)

Fidelity: [0.999]
Angles: [0.08699895 0.01804698 0.68801853 0.37326585 1.63846585 1.40133293]


In [8]:
#d) Bars or Stripes - Lets use it to check
def printDiagram(state):
    out = "Qubits:\n"
    for i in range(4):
        if i in state: out += "1 "
        else: out += "0 "
        if i == 1: out += "\n"
    print(out)
    
def barsStripes(qubits):
    gatelist = []
    side = []
    bars = [[1, 3], [0, 2]]
    stripes = [[2, 3], [0, 1]]
    
    parity = True#Goal is the target state of objective
    choice = random.random()
    if random.random() < 0.5:
        side = random.choice(bars)
        parity = False
    else:
        side = random.choice(stripes)

    printDiagram(side)
    #Initial Gates
    for i in side:
        gatelist.append(cirq.X(qubits[i]))
        
    return gatelist, parity

def mapParity(angles, gates, qubits, anc):
    gatelist = []
    #Single Qubit Rotations
    gatelist.append(gates[0](angles[0])(qubits[0]))
    gatelist.append(gates[1](angles[1])(qubits[1]))
    gatelist.append(gates[2](angles[2])(qubits[2]))
    gatelist.append(gates[3](angles[3])(qubits[3]))
    gatelist.append(gates[4](angles[4])(anc))
    
    #Entangling Gates - these were not chosen randomly, this directly encodes the problem into the ancilla
    #For the record, I came up with this idea and it is basically cheating
    gatelist.append(cirq.CNOT(qubits[0], qubits[1]))
    gatelist.append(cirq.CNOT(qubits[1], anc))#q1 is 0: 01 or 23, 1: 02 or 13s
    #0 1  stripes: 0 0  1 1   bars:  1 0   0 1
    #2 3           1 1  0 0          1 0   0 1
    
    #Readout Rotation
    gatelist.append(gates[5](angles[5])(anc))
    #Measure
    gatelist.append(cirq.measure(anc, key = 'z'))
    return gatelist

def measureCircuit(gatelist):
    sim = cirq.Simulator()
    circuit = cirq.Circuit(gatelist)
    sample = sim.run(circuit)
    result = int(str(sample)[2])
    return result

In [9]:
for i in range(6):
    oplist, parity = barsStripes(qubits)
    oplist.append(mapParity(angles, plist, qubits, anc))
    predict = measureCircuit(oplist)

    if predict == 1:
        print("Prediction: Horizontal\n")
    else:
        print("Prediction: Vertical\n")

Qubits:
1 0 
1 0 
Prediction: Vertical

Qubits:
0 0 
1 1 
Prediction: Horizontal

Qubits:
0 1 
0 1 
Prediction: Vertical

Qubits:
1 1 
0 0 
Prediction: Horizontal

Qubits:
0 0 
1 1 
Prediction: Horizontal

Qubits:
1 1 
0 0 
Prediction: Horizontal

